## Installation

In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb

## Import the model

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [10]:
# used to decrease size of chunk
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [14]:
# use to process the data with the RoBERTa model
def batch_qqp(data, batch_size=128):
    ret = []
    for d in chunks(data, batch_size):
        t = tokenizer([a[0] for a in d], [a[1] for a in d], return_tensors='pt', padding=True)
        # t = tokenizer([a[0] for a in d], [a[1] for a in d], return_tensors='pt', padding=True).to('cuda')
        with torch.no_grad():
            logits = torch.softmax(model(**t)[0], dim=1).numpy()
            # logits = torch.softmax(model(**t)[0], dim=1).cpu().numpy()
        ret.append(logits)
    return np.vstack(ret)
    

In [15]:
from checklist.pred_wrapper import PredictorWrapper
# wrapped_pp returns a tuple with (predictions, softmax confidences)
wrapped_pp = PredictorWrapper.wrap_softmax(batch_qqp) #to make sure that the output of the function will be formatted

In [16]:
# Test the output
s0 = "The company HuggingFace is based in New York City"
s1 = "Apples are especially bad for your health"
s2 = "HuggingFace's headquarters are situated in Manhattan"
batch_qqp([(s0, s1), (s0, s2)]) 

array([[0.94038326, 0.05961677],
       [0.09536293, 0.90463704]], dtype=float32)

In [17]:
suite_path = 'qqp_suite.pkl' #define path of the pkl
suite = TestSuite.from_file(suite_path)

In [28]:
#run the test with 10 test case. I have tried or n=100. It spend time for many hours .....
# overwrite = True which meant write over the old file
suite.run(wrapped_pp, n=10, overwrite=True) 


Running Modifier: adj
Predicting 10 examples
Running different adjectives
Predicting 10 examples
Running Different animals
Predicting 10 examples
Running Irrelevant modifiers - animals
Predicting 10 examples
Running Irrelevant modifiers - people
Predicting 10 examples
Running Irrelevant preamble with different examples.
Predicting 10 examples
Running Preamble is relevant (different injuries)
Predicting 10 examples
Running How can I become more {synonym}?
Predicting 10 examples
Running (question, f(question)) where f(question) replaces synonyms?
Predicting 10 examples
Running Replace synonyms in real pairs
Predicting 28 examples
Running How can I become more X != How can I become less X
Predicting 10 examples
Running How can I become more X = How can I become less antonym(X)
Predicting 10 examples
Running add one typo
Predicting 30 examples
Running contrations
Predicting 28 examples
Running (q, paraphrase(q))
Predicting 928 examples
Running Product of paraphrases(q1) * paraphrases(q2)
P

In [29]:
suite.summary() #summary the test case

Vocabulary

Modifier: adj
Test cases:      1000
Test cases run:  10
Fails (rate):    10 (100.0%)

Example fails:
0.9 ('Is Harriet Clarke an accountant?', 'Is Harriet Clarke a good accountant?')
----
0.9 ('Is Susan Crawford a kid?', 'Is Susan Crawford an actual kid?')
----
0.9 ('Is Kevin Shaw an author?', 'Is Kevin Shaw an amazing author?')
----


different adjectives
Test cases:      963
Test cases run:  10
Fails (rate):    10 (100.0%)

Example fails:
0.9 ('Is Wendy Wright Jewish?', 'Is Wendy Wright an immigrant?')
----
0.9 ('Is Eleanor Nelson American?', 'Is Eleanor Nelson an anarchist?')
----
0.9 ('Is Mike White Australian?', 'Is Mike White an inventor?')
----


Different animals
Test cases:      942
Test cases run:  10
Fails (rate):    10 (100.0%)

Example fails:
0.9 ('Can I feed my lobster bananas?', 'Can I feed my lizard bananas?')
----
0.9 ('Can I feed my chicken formula?', 'Can I feed my turtle formula?')
----
0.9 ('Can I feed my snake seeds?', 'Can I feed my owl seeds?')
----

